In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 58.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.9/92.9 kB 13.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.7/302.7 kB 32.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.7/138.7 kB 21.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 381.9/381.9 kB 34.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.7/59.7 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 18.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 44.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 kB 7.5 MB

# If you have a problem with running the python notebook i.e. it throws errors, please restart runtime. Please ensure you are using the GPU before running for a smooth experience

In [ ]:
# !pip install -q gradio

In [ ]:
# !pip install typing-extensions --upgrade

In [ ]:
import pandas as pd
import math
from sklearn.metrics import mean_squared_error
import random
from datetime import datetime as dt
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import numpy as np
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.arima.model import ARIMA
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM
from tensorflow.keras.layers import BatchNormalization
import gradio as gr
import matplotlib.pyplot as plt
import plotly.graph_objects as go

def predict_stock_price_ARIMA(Stock_name):
    Stock_name1 = '/content/drive/MyDrive/ME781/'+ Stock_name.lower()
    df = pd.read_csv(f'{Stock_name1}.csv')

    # Converting object datatype into datetime datatype
    df2 = pd.DataFrame(df['Close'])
    df2['Date'] = pd.to_datetime(df['Date']).dt.date

    df2.set_index('Date', inplace=True)

    first_diffs = df2.Close - df2.Close.shift(1)

    # Applying transformation to make data statinoary
    df2['FirstDifference'] = first_diffs
    df2.dropna(inplace=True)

    # Creating the ARIMA model and fitting the values
    model = ARIMA(df2.FirstDifference.dropna(), order=(0,1,0))
    result = model.fit()

    residual = result.resid
    residual = residual.cumsum()

    # Inversing the transformation to get original data back
    a = df2.Close - df2.FirstDifference
    diff_results = residual + residual.shift(-1)
    diff_results = diff_results + a

    df1 = diff_results
    df1.dropna(inplace=True)

    # Normalizing the data
    scaler = MinMaxScaler(feature_range=(0,1))
    df1 = scaler.fit_transform(np.array(df1).reshape(-1,1))

    df1 = df1.reshape(len(df1))

    # Defining the train and test data size
    training_size=int(len(df1)*0.9)
    test_size = len(df1)-training_size

    # convert an array of values into a dataset matrix
    def create_dataset(dataset, time_step=1):
        dataX, dataY = [], []
        for i in range(len(dataset)-time_step-1):
            a = dataset[i:(i+time_step)]   ###i=0, 0,1,2,3-----99   100
            dataX.append(a)
            dataY.append(dataset[i + time_step])
        return np.array(dataX), np.array(dataY)

    time_step = 60
    X_train, y_train = create_dataset(df1[:training_size], time_step)
    X_test, y_test = create_dataset(df1[training_size:], time_step)

    X_train =X_train.reshape(X_train.shape[0],X_train.shape[1] , 1)
    X_test = X_test.reshape(X_test.shape[0],X_test.shape[1] , 1)

    # Creating LSTM model
    model=Sequential()
    model.add(LSTM(30, input_shape=(X_train.shape[1],1)))
    model.add(Dropout(0.3))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error',optimizer='adam', metrics=['accuracy'])

    # Fiting data into LSTM model
    model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=5,batch_size=64,verbose=1)

    # Predicting the values from fitted data
    train_predict=model.predict(X_train)
    test_predict= model.predict(X_test)

    # Calculating root mean square error
    rmse = math.sqrt(mean_squared_error(y_test,test_predict))

    final_test = pd.DataFrame(df2[len(df2) - len(test_predict):])

    # Un-normalizing the data
    final_test['test'] = scaler.inverse_transform(test_predict)

    # Plotting the predictied price wrt original price
    plt.figure(figsize=(20,8))
    plt.plot(final_test.Close, label='Original Price', color='red')
    plt.plot(final_test.test, label='Predicted Price', color='green')
    plt.title('Stock of '+Stock_name, fontweight='bold', fontsize=20)
    plt.ylabel('Price in Thousand $', fontweight='bold', fontsize=20)
    plt.xticks(rotation=45)
    plt.legend(prop={'size':25})

    return rmse, plt

def predict_stock_priceLSTM(Stock_name):
    # df = pd.read_csv(f'{Stock_name}.csv')
    Stock_name1 = '/content/drive/MyDrive/ME781/'+ Stock_name.lower()
    df = pd.read_csv(f'{Stock_name1}.csv')
    # Dropping the rows with null values
    df = df.dropna()

    # Creating an array of close values and normalising them
    df_close = df.Close.values.reshape(-1,1)
    scaler = MinMaxScaler(feature_range=(0, 1))
    df_close = scaler.fit_transform(df_close)

    # Splitting the data into testing and training sets
    train_size = 0.9
    training_size=int(len(df_close)*train_size)
    test_size=len(df_close)-training_size
    train_data,test_data=df_close[0:training_size,:],df_close[training_size:len(df_close),:1]

    # Creating batches of dataset of length = timestep
    def create_dataset(df, timestep):
        df_X, df_y = [], []
        for i in range(len(df) - timestep):
            df_X.append(df[i:i + timestep, 0])
            df_y.append(df[i + timestep, 0])
        return np.array(df_X), np.array(df_y)

    # Sliding window length= 60 days : Hyperparameter
    timestep=60

    X_train_lstm, y_train_lstm = create_dataset(train_data, timestep)
    X_test_lstm, y_test_lstm = create_dataset(test_data, timestep)

    X_train_lstm = X_train_lstm.reshape(X_train_lstm.shape[0],X_train_lstm.shape[1] , 1)
    X_test_lstm = X_test_lstm.reshape(X_test_lstm.shape[0],X_test_lstm.shape[1] , 1)

    vanilla_lstm = Sequential([
        LSTM(units = 128, return_sequences = True, input_shape = (timestep, 1)),
        Dropout(rate = 0.15),

        LSTM(units = 128, return_sequences = True),
        LSTM(units = 128),
        Dense(units = 1)
    ])
    vanilla_lstm.compile(optimizer = 'adam', loss = 'mean_squared_error')

    # tensorflow.keras.utils.plot_model(vanilla_lstm)

    from tensorflow.keras.callbacks import EarlyStopping
    early_stop = EarlyStopping(monitor = 'loss', patience = 1)
    vanilla_lstm.fit(X_train_lstm, y_train_lstm, validation_data = (X_test_lstm, y_test_lstm), epochs = 50, batch_size = 64, callbacks = [early_stop])

    train_pred = scaler.inverse_transform(vanilla_lstm.predict(X_train_lstm))
    y_train_lstm = scaler.inverse_transform(y_train_lstm.reshape(-1,1))

    test_pred = scaler.inverse_transform(vanilla_lstm.predict(X_test_lstm))
    y_test_lstm = scaler.inverse_transform(y_test_lstm.reshape(-1,1))

    mse_test = mean_squared_error(y_test_lstm, test_pred)
    rmse_test = np.sqrt(mse_test)

    #Restrcuting the output to plot by date
    dates = df['Date'][df.shape[0] - y_test_lstm.shape[0]:]
    data = zip(dates.tolist(), np.reshape(y_test_lstm, y_test_lstm.shape[0]), np.reshape(test_pred, test_pred.shape[0]))
    d = pd.DataFrame(data, columns = ['Dates', 'Actual', 'Predicted'] )
    d.set_index('Dates', inplace = True)

    plt.figure(figsize = (20,8))
    plt.plot(d.Actual, label='Original Price', color = 'red')
    plt.plot(d.Predicted, label='Predicted Price', color = 'blue')
    plt.title(f'Stock of {Stock_name}', fontweight='bold', fontsize=20)
    plt.ylabel('Price in Thousand $', fontweight='bold', fontsize=20)
    plt.xticks(np.arange(0, d.Predicted.shape[0], 100), rotation = 45)
    plt.legend(prop={'size':25})

    return rmse_test, plt

def predict_stock_price(Model_name, Stock_name):
    if Model_name == 'LSTM':
        return predict_stock_priceLSTM(Stock_name)
    elif Model_name == 'ARIMA_with_LSTM':
        return predict_stock_price_ARIMA(Stock_name)
    else:
        return "Invalid model name. Please enter either 'LSTM' or 'ARIMA_with_LSTM'."


iface = gr.Interface(fn=predict_stock_price, inputs=["text", "text"],
                     outputs=["text", "plot"],
                     title="Stock Price Predictor: Choose from over 500 stocks to predict",
                     description="Enter the name of the stock to predict and the model to use (either 'LSTM' or 'ARIMA_with_LSTM'). The S&P 500 Index comprises 503 stocks, data for which is available online. S&P provides contractions for each company's stocks for convenience: For Example, Google is GOOGL, Amazon is AMZN. Please find a list of these contractions at: https://en.wikipedia.org/wiki/List_of_S%26P_500_companies. Please enter the S&P contracted stock name for predicting the stock of your choice",
                     examples=[["LSTM", "IBM"], ["ARIMA_with_LSTM", "GOOGL"], ["Output 0", "Root Mean Square Test Error"], ["Output 1", "Predicted Stock Prices over a period of time"]])
iface.launch(debug = False)

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://6d3a85ff877314a4e5.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
